### This Script Plots the Ablation Curves

import relevant Libraries

In [ ]:
%pip install pyarrow
%pip install plotly
%pip install feature_engine
%pip update pandas

In [2]:
# Base libraries
import pandas as pd
import numpy as np
import os
import math as math
import datetime
from scipy import stats

# Visualizations
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go

import seaborn as sns

In [3]:
import os
from pyrsistent import v
os.getcwd()
#os.chdir('/Users/asgerlyngeholst-hansen/Desktop/GEUS-Master-Thesis/')
#os.chdir('C:\\Users\\nifu18ab\\Desktop\\GEUS-Master-Thesis')
os.chdir('/Users/nilsfulde/Desktop/GEUS-Master-Thesis')
#os.chdir('/Users/nilsfulde/Desktop/Master_Thesis')

Load new_promice_ablation Data

In [5]:
#Function for loading data and converting to datetime 
def load_promice(path_to_file): 
    '''' Loading PROMICE data for a given path into a DataFrame. + adding time index

    INTPUTS:
        path_to_file: Path to the desired file containing PROMICE data [string]

    OUTPUTS:
        df: Dataframe containing PROMICE data for the desired settings [DataFrame]
    '''

    df = pd.read_parquet(path_to_file, engine='pyarrow')
    df['time'] = pd.to_datetime(df.time, utc=True)
    df = df.set_index('time')
    return df
    df = load_promice('data/new_promice/all_promice_data_hourly.parquet.gzip')

In [6]:
df = load_promice('data/new_promice/all_promice_data_hourly.parquet.gzip')

Plotting Function

In [1]:
# This bit to plot the Melting seasons
def plt_srf_height(Station, input_df = df, y = df['Surface height from combined measurements'], Period = "All" ):
    
    df = input_df
    
    df = df[df['stid'] == Station].copy() 
    
    if Period == "All":
        df = df
    else:
        start_date = Period[0]
        end_date = Period[1]
        df = df[df['Datetime'].between(start_date,end_date)]
    
    # Define the colors for the melting season categories
    colors = {'': 'black', 'beginning': 'green', 'middle': 'orange', 'end': 'red'}
        
    # Define the x-axis and y-axis values
    x = df['Datetime']
    y = y

    # Create a figure and axis object
    fig, ax = plt.subplots(figsize=(25, 10))

    # Iterate over the melting season categories
    for category in ['', 'beginning', 'middle', 'end']:

        # Get the indices where the Melting Season column matches the current category
        indices = df[df['Melting Season'] == category].index

        # Plot a scatter plot for the current category
        ax.scatter(x[indices], y[indices], color=colors[category], alpha=0.7, label=category)

    # Add a legend to the plot
    ax.legend()

    # Shade areas with "Ablation" = True
    ablation_mask = df["Ablation"]
    ablation_starts = df["Datetime"][ablation_mask & ~ablation_mask.shift(1, fill_value=False)]
    ablation_ends = df["Datetime"][ablation_mask & ~ablation_mask.shift(-1, fill_value=False)]
    for start, end in zip(ablation_starts, ablation_ends):
        ax.axvspan(start, end, color="yellow", alpha=0.2)

    # Set the title and axis labels
    ax.set_title(pd.DataFrame(y).columns.to_list()[0])
    ax.set_xlabel('Date')
    ax.set_ylabel('Surface Height (m)')

    # Show the plot
    plt.show()

NameError: name 'df' is not defined

In [ ]:
plt_srf_height(Station = "KAN_M", input_df = df) 